## Definindo o Problema

In [12]:
#!pip install Catboost;
#!pip install pymoo;
#!pip install -U pyrecorder;

In [13]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from yellowbrick.classifier import ConfusionMatrix
import numpy as np
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
#from google.colab import drive
import pandas as pd
import pickle
from pymoo.algorithms.soo.nonconvex.pso import PSO, PSOAnimation
#from pymoo.factory import Rastrigin
from pymoo.optimize import minimize
import matplotlib.pyplot as plt
from pymoo.factory import get_termination
#from pymoo.util.display import Display
from pymoo.core.callback import Callback

In [14]:
from catboost import CatBoostClassifier

In [15]:
#drive.mount('/content/gdrive')

In [16]:
#with open('/content/gdrive/MyDrive/datasets/credit.pkl', 'rb') as f:
#    x_wine_train, x_wine_test, y_wine_train, y_wine_test = pickle.load(f)
    
#with open('./wine.pkl', 'rb') as f:
#    x_wine, y_wine = pickle.load(f)
    
with open('./wine.pkl', 'rb') as f:
    x_wine, y_wine = pickle.load(f)

Definindo as restrições 

In [17]:
DIMENSIONS = 24

In [18]:
iterations_min = 20  #n_estimators n_estimators #INT
learning_rate_min = 0.001  #FLOAT
l2_leaf_reg_min = 0.01  #FLOAT reg_lambda
bagging_temperature_min = 0.001  # float
subsample_min = 1.0   #float 
mvs_reg_min = 0.001  #float 0 - inf
random_strength_min = 0.001  #float 1.0 
use_best_model_min = False  #bool true false
best_model_min_trees_min = 1  #int 
depth_min = 1  #max_depth 8 int
min_data_in_leaf_min = 1  #min_child_samples int
max_leaves_min = 10  #num_leaves int 
one_hot_max_size_min = 1  #int 
rsm_min = 0.001  #colsample_bylevel float (0;1]
fold_permutation_block_min = 1  #int 
leaf_estimation_iterations_min = 1  #int
fold_len_multiplier_min = 2.0  #float
approx_on_full_history_min = False  #bool 
boost_from_average_min = False  #bool 
diffusion_temperature_min = 1000  #float 10000
posterior_sampling_min = False  #bool
allow_const_label_min = False  #bool
penalties_coefficient_min = 1.0  #float
model_shrink_rate_min = 0.001  #float

iterations_max = 100  #n_estimators n_estimators #INT
learning_rate_max = 0.5  #FLOAT
l2_leaf_reg_max = 0.01  #FLOAT reg_lambda
bagging_temperature_max = 10000.0  # float
subsample_max = 2.0  #float 
mvs_reg_max = 1000.0  #float 0 - inf
random_strength_max = 10.0  #float 1.0 
use_best_model_max = True  #bool true false
best_model_min_trees_max = 10  #int 
depth_max = 1  #max_depth 8 int 
min_data_in_leaf_max = 100  #min_child_samples int
max_leaves_max = 10  #num_leaves int 
one_hot_max_size_max = 20  #int 
rsm_max = 1.0   #colsample_bylevel float (0;1]
fold_permutation_block_max = 100  #int 
leaf_estimation_iterations_max = 100  #int
fold_len_multiplier_max = 10.0  #float
approx_on_full_history_max = True  #bool 
boost_from_average_max = True  #bool 
diffusion_temperature_max = 50000  #float 10000
posterior_sampling_max = True  #bool
allow_const_label_max = True  #bool
penalties_coefficient_max = 10.0  #float
model_shrink_rate_max = 1.0  #float

grow_policy = 'Lossguide'

Definindo o problema

In [19]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class OptimizeWithAccuracy(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([iterations_min,
                          learning_rate_min,
                          l2_leaf_reg_min,
                          bagging_temperature_min,
                          subsample_min,
                          mvs_reg_min,
                          random_strength_min,
                          use_best_model_min,
                          best_model_min_trees_min,
                          depth_min,
                          min_data_in_leaf_min,
                          max_leaves_min,
                          one_hot_max_size_min,
                          rsm_min,
                          fold_permutation_block_min,
                          leaf_estimation_iterations_min,
                          fold_len_multiplier_min,
                          approx_on_full_history_min,
                          boost_from_average_min,
                          diffusion_temperature_min,
                          posterior_sampling_min,
                          allow_const_label_min,
                          penalties_coefficient_min,
                          model_shrink_rate_min]),
                         xu=np.array([iterations_max,
                            learning_rate_max,
                            l2_leaf_reg_max,
                            bagging_temperature_max,
                            subsample_max,
                            mvs_reg_max,
                            random_strength_max,
                            use_best_model_max,
                            best_model_min_trees_max,
                            depth_max,
                            min_data_in_leaf_max,
                            max_leaves_max,
                            one_hot_max_size_max,
                            rsm_max,
                            fold_permutation_block_max,
                            leaf_estimation_iterations_max,
                            fold_len_multiplier_max,
                            approx_on_full_history_max,
                            boost_from_average_max,
                            diffusion_temperature_max,
                            posterior_sampling_max,
                            allow_const_label_max,
                            penalties_coefficient_max,
                            model_shrink_rate_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_catboost = CatBoostClassifier(#num_leaves        = int(np.round(x[0])), 
                                       #extra_trees = bool(np.round(x[16])), #boolean
                                        iterations = int(np.round(x[0])),
                                        
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        #subsample = x[4],
                                        #mvs_reg = x[5],
                                        random_strength = x[6],
                                        #use_best_model = bool(np.round(x[7])),
                                        best_model_min_trees = int(np.round(x[8])), 
                                        depth = int(np.round(x[9])), 
                                        min_data_in_leaf = int(np.round(x[10])), 
                                        max_leaves = int(np.round(x[11])), 
                                        one_hot_max_size = int(np.round(x[12])), 
                                        rsm = x[13],
                                        fold_permutation_block = int(np.round(x[14])), 
                                        leaf_estimation_iterations = int(np.round(x[15])), 
                                        fold_len_multiplier = x[16],
                                        approx_on_full_history = False, #bool(np.round(x[17])),
                                        #boost_from_average = bool(np.round(x[18])),
                                        #diffusion_temperature = x[19],
                                        posterior_sampling = bool(np.round(x[20])),
                                        allow_const_label = bool(np.round(x[21])),
                                        penalties_coefficient = x[22],
                                        model_shrink_rate = x[23],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                       )
        
        kfold = KFold(n_splits = 3, shuffle = True)
        
        scores = cross_val_score(model_catboost, x_wine, y_wine, cv = kfold, n_jobs=-1)  
        result = scores.mean()
        #if not result:
        #  result = 0
        
        out['F'] = -1 * result

problemAccuracy = OptimizeWithAccuracy()

In [20]:
class OptimizeWithF1(ElementwiseProblem):
        def __init__(self):
                super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([iterations_min,
                          learning_rate_min,
                          l2_leaf_reg_min,
                          bagging_temperature_min,
                          subsample_min,
                          mvs_reg_min,
                          random_strength_min,
                          use_best_model_min,
                          best_model_min_trees_min,
                          depth_min,
                          min_data_in_leaf_min,
                          max_leaves_min,
                          one_hot_max_size_min,
                          rsm_min,
                          fold_permutation_block_min,
                          leaf_estimation_iterations_min,
                          fold_len_multiplier_min,
                          approx_on_full_history_min,
                          boost_from_average_min,
                          diffusion_temperature_min,
                          posterior_sampling_min,
                          allow_const_label_min,
                          penalties_coefficient_min,
                          model_shrink_rate_min]),
                         xu=np.array([iterations_max,
                            learning_rate_max,
                            l2_leaf_reg_max,
                            bagging_temperature_max,
                            subsample_max,
                            mvs_reg_max,
                            random_strength_max,
                            use_best_model_max,
                            best_model_min_trees_max,
                            depth_max,
                            min_data_in_leaf_max,
                            max_leaves_max,
                            one_hot_max_size_max,
                            rsm_max,
                            fold_permutation_block_max,
                            leaf_estimation_iterations_max,
                            fold_len_multiplier_max,
                            approx_on_full_history_max,
                            boost_from_average_max,
                            diffusion_temperature_max,
                            posterior_sampling_max,
                            allow_const_label_max,
                            penalties_coefficient_max,
                            model_shrink_rate_max])
                        )

        def _evaluate(self, x, out, *args, **kwargs):
                #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
                
                model_catboost = CatBoostClassifier(#num_leaves        = int(np.round(x[0])), 
                                        #extra_trees = bool(np.round(x[16])), #boolean
                                                iterations = int(np.round(x[0])),
                                        
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        #subsample = x[4],
                                        #mvs_reg = x[5],
                                        random_strength = x[6],
                                        #use_best_model = bool(np.round(x[7])),
                                        best_model_min_trees = int(np.round(x[8])), 
                                        depth = int(np.round(x[9])), 
                                        min_data_in_leaf = int(np.round(x[10])), 
                                        max_leaves = int(np.round(x[11])), 
                                        one_hot_max_size = int(np.round(x[12])), 
                                        rsm = x[13],
                                        fold_permutation_block = int(np.round(x[14])), 
                                        leaf_estimation_iterations = int(np.round(x[15])), 
                                        fold_len_multiplier = x[16],
                                        approx_on_full_history = False, #bool(np.round(x[17])),
                                        #boost_from_average = bool(np.round(x[18])),
                                        #diffusion_temperature = x[19],
                                        posterior_sampling = bool(np.round(x[20])),
                                        allow_const_label = bool(np.round(x[21])),
                                        penalties_coefficient = x[22],
                                        model_shrink_rate = x[23],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                        )
                
                kfold = KFold(n_splits = 3, shuffle = True)

                scores = cross_val_score(model_catboost, x_wine, y_wine, cv = kfold, scoring='f1_weighted', n_jobs=-1)  
                
                result = scores.mean()
                out['F'] = -1 * result

problemF1 = OptimizeWithF1()

In [21]:
class OptimizeWithAUC(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([iterations_min,
                          learning_rate_min,
                          l2_leaf_reg_min,
                          bagging_temperature_min,
                          subsample_min,
                          mvs_reg_min,
                          random_strength_min,
                          use_best_model_min,
                          best_model_min_trees_min,
                          depth_min,
                          min_data_in_leaf_min,
                          max_leaves_min,
                          one_hot_max_size_min,
                          rsm_min,
                          fold_permutation_block_min,
                          leaf_estimation_iterations_min,
                          fold_len_multiplier_min,
                          approx_on_full_history_min,
                          boost_from_average_min,
                          diffusion_temperature_min,
                          posterior_sampling_min,
                          allow_const_label_min,
                          penalties_coefficient_min,
                          model_shrink_rate_min]),
                         xu=np.array([iterations_max,
                            learning_rate_max,
                            l2_leaf_reg_max,
                            bagging_temperature_max,
                            subsample_max,
                            mvs_reg_max,
                            random_strength_max,
                            use_best_model_max,
                            best_model_min_trees_max,
                            depth_max,
                            min_data_in_leaf_max,
                            max_leaves_max,
                            one_hot_max_size_max,
                            rsm_max,
                            fold_permutation_block_max,
                            leaf_estimation_iterations_max,
                            fold_len_multiplier_max,
                            approx_on_full_history_max,
                            boost_from_average_max,
                            diffusion_temperature_max,
                            posterior_sampling_max,
                            allow_const_label_max,
                            penalties_coefficient_max,
                            model_shrink_rate_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_catboost = CatBoostClassifier(#num_leaves        = int(np.round(x[0])), 
                                       #extra_trees = bool(np.round(x[16])), #boolean
                                        iterations = int(np.round(x[0])),
                                        
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        #subsample = x[4],
                                        #mvs_reg = x[5],
                                        random_strength = x[6],
                                        #use_best_model = bool(np.round(x[7])),
                                        best_model_min_trees = int(np.round(x[8])), 
                                        depth = int(np.round(x[9])), 
                                        min_data_in_leaf = int(np.round(x[10])), 
                                        max_leaves = int(np.round(x[11])), 
                                        one_hot_max_size = int(np.round(x[12])), 
                                        rsm = x[13],
                                        fold_permutation_block = int(np.round(x[14])), 
                                        leaf_estimation_iterations = int(np.round(x[15])), 
                                        fold_len_multiplier = x[16],
                                        approx_on_full_history = False, #bool(np.round(x[17])),
                                        #boost_from_average = bool(np.round(x[18])),
                                        #diffusion_temperature = x[19],
                                        posterior_sampling = bool(np.round(x[20])),
                                        allow_const_label = bool(np.round(x[21])),
                                        penalties_coefficient = x[22],
                                        model_shrink_rate = x[23],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                       )
        
        kfold = KFold(n_splits = 3, shuffle = True)
   
        scores = cross_val_score(model_catboost, x_wine, y_wine, cv = kfold, scoring='roc_auc_ovr_weighted', n_jobs=-1)  
        
        result = scores.mean()

        out['F'] = -1 * result

problemAUC = OptimizeWithAUC()

In [22]:
from pymoo.util.display.column import Column
from pymoo.util.display.output import Output

ModuleNotFoundError: No module named 'pymoo.util.display.column'; 'pymoo.util.display' is not a package

In [12]:
class MyOutput(Output):

    def __init__(self):
        super().__init__()
        global pbar 
        pbar = tqdm(total=ITERATIONS)
        self.score = Column("score", width=13)
        self.Parameters = Column("Parameters", width=35)
        self.columns += [self.score, self.Parameters]

    def update(self, algorithm):
        super().update(algorithm)
        self.score.set(-np.min(algorithm.pop.get("F")))
        #self.Parameters.set(algorithm.pop.get("X")[0])
        pbar.update(1)
        if pbar.n == ITERATIONS: pbar.close()

## Particle Swarm Optimization (PSO)

### Acurácia

In [5]:
!pip install pyswarms

  Using cached pyswarms-1.3.0-py2.py3-none-any.whl (104 kB)
     ------------------------------------- 153.2/153.2 kB 3.0 MB/s eta 0:00:00
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)


In [9]:
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
import numpy as np

In [23]:
xl=np.array([iterations_min,
            learning_rate_min,
            l2_leaf_reg_min,
            bagging_temperature_min,
            random_strength_min,
            best_model_min_trees_min,
            depth_min,
            min_data_in_leaf_min,
            max_leaves_min,
            one_hot_max_size_min,
            rsm_min,
            fold_permutation_block_min,
            leaf_estimation_iterations_min,
            fold_len_multiplier_min,
            penalties_coefficient_min,
            model_shrink_rate_min])
xu=np.array([iterations_max,
            learning_rate_max,
            l2_leaf_reg_max,
            bagging_temperature_max,
            random_strength_max,
            best_model_min_trees_max,
            depth_max,
            min_data_in_leaf_max,
            max_leaves_max,
            one_hot_max_size_max,
            rsm_max,
            fold_permutation_block_max,
            leaf_estimation_iterations_max,
            fold_len_multiplier_max,
            penalties_coefficient_max,
            model_shrink_rate_max])

In [26]:
# Call an instance of PSO
swarm_size = 32
iters = 32
dim = 16
options = {'c1': 1.5, 'c2':1.5, 'w':0.5}
constraints = (xl,xu)

optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                    dimensions=dim,
                                    options=options,
                                    bounds=constraints)



In [32]:
def PSO_Optimize_Accuracy(x):
    print(x)

    return -result

In [31]:
# Perform optimization
cost, joint_vars = optimizer.optimize(PSO_Optimize_Accuracy, iters=iters)

2023-04-12 07:38:06,932 - pyswarms.single.global_best - INFO - Optimize for 32 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
pyswarms.single.global_best:   0%|          |0/32


TypeError: only size-1 arrays can be converted to Python scalars

In [13]:
ITERATIONS = 32
POPULATION = 32

In [14]:
def run_accuracy_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(#pop_size=POPULATION, 
                    #w=0.3, c1=4.0, c2=1.0, 
                    #adaptive=False, initial_velocity='random', 
                    #max_velocity_rate=0.75, pertube_best=False,
                    max_velocity_rate=0.25,
                    pop_size=POPULATION)
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAccuracy,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best Accuracy Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    return score_best_individual, parameters_best_individual, res

In [15]:
score_best_individual, parameters_best_individual, res = run_accuracy_pso(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_19812\2673210061.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [16]:
#final_model = lgb.LGBMClassifier(num_leaves       = int(np.round(parameters_best_individual[0])), 
#                                min_child_samples = int(np.round(parameters_best_individual[1])), 
#                                n_estimators      = int(np.round(parameters_best_individual[2])),
#                                learning_rate     = parameters_best_individual[3] ,
#                                subsample_for_bin = int(np.round(parameters_best_individual[4])),
#                                min_split_gain    = parameters_best_individual[5],
#                                min_child_weight  = parameters_best_individual[6],
#                                reg_alpha         = parameters_best_individual[7],
#                                reg_lambda        = parameters_best_individual[8],
#                                max_depth=- 1
#                                )
#
#
#kfold = KFold(n_splits = 3, shuffle = True)
#
#scores = cross_val_score(final_model, x_wine, y_wine, cv = kfold, n_jobs=-1)  
#
#result = scores.mean()
###print(result)

In [17]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 

In [18]:
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [19]:
#Accuracy_PSO = scorelist

In [20]:
#plt.plot(Accuracy_PSO);

In [21]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_PSO_Accuracy = tracking
#plt.plot(tracking)

### F1 Score 

In [22]:
def run_f1_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(#pop_size=POPULATION, 
                    #w=0.3, c1=4.0, c2=1.0, 
                    #adaptive=False, initial_velocity='random', 
                    #max_velocity_rate=0.75, pertube_best=False,
                    max_velocity_rate=0.25,
                    pop_size=POPULATION)
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemF1,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best F1 Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    return score_best_individual, parameters_best_individual, res

In [23]:
score_best_individual, parameters_best_individual, res = run_f1_pso(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_19812\3496873573.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [24]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#F1_PSO = scorelist
#
#plt.plot(scorelist);

In [25]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_PSO_F1 = tracking
#plt.plot(tracking)

### AUC

In [26]:
def run_auc_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(#pop_size=POPULATION, 
                    #w=0.3, c1=4.0, c2=1.0, 
                    #adaptive=False, initial_velocity='random', 
                    #max_velocity_rate=0.75, pertube_best=False,
                    max_velocity_rate=0.25,
                    pop_size=POPULATION)
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAUC,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best AUC Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [59]:
score_best_individual, parameters_best_individual, res = run_auc_pso(ITERATIONS = 15, POPULATION = 5)

C:\Users\Felps\AppData\Local\Temp\ipykernel_19812\1707635262.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\Felps\anaconda3\envs\pymoo\lib\site-packages\pymoo\util\misc.py:99: RuntimeWarning: invalid value encountered in divide
  return lambda a, b: np.sqrt((((a - b) / (xu - xl)) ** 2).sum(axis=1))


TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'

In [28]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#AUC_PSO = scorelist
#
#plt.plot(scorelist);

In [29]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_PSO_AUC = tracking
#plt.plot(tracking)

## Algoritmo Genético (GA)

### Acurácia

In [30]:
ITERATIONS = 32
POPULATION = 32

In [31]:
from pymoo.algorithms.soo.nonconvex.ga import GA

In [32]:
def run_accuracy_ga(ITERATIONS = 32, POPULATION = 32):

    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAccuracy,
                algorithm,
                save_history=False,
                verbose=True,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best Accuracy Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [33]:
score_best_individual, parameters_best_individual, res = run_accuracy_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_19812\2209151331.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

n_gen  |  n_eval  |     score     |              Parameters            
     1 |        1 |  0.2203686533 |                                   -


In [34]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_Accuracy = tracking
#plt.plot(tracking)

In [35]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 

In [36]:
#scorelist = []
#
#for i in range(len(trlist)):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [37]:
#Accuracy_GA = scorelist

In [38]:
#plt.plot(scorelist);

### F1 Score 

In [39]:
from pymoo.algorithms.soo.nonconvex.ga import GA
def run_f1_ga(ITERATIONS = 32, POPULATION = 32):
    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemF1,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best F1 Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [40]:
score_best_individual, parameters_best_individual, res = run_f1_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_19812\1994123235.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [41]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#F1_GA = scorelist
#
#plt.plot(scorelist);

In [42]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_F1 = tracking
#plt.plot(tracking)

### AUC

In [43]:
from pymoo.algorithms.soo.nonconvex.ga import GA
def run_auc_ga(ITERATIONS = 32, POPULATION = 32):
    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAUC,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best AUC Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [44]:
score_best_individual, parameters_best_individual, res = run_auc_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_19812\3250583479.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [45]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#AUC_GA = scorelist
#
#plt.plot(scorelist);

In [46]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_AUC = tracking
#plt.plot(tracking)

## Grid Search

### Acurácia

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
#n_possibilities = 2
#
##num_leaves_grid = [i for i in range(num_leaves_min,num_leaves_max, int((num_leaves_max)/13))]
#num_leaves_grid = [i for i in map(lambda x: int(x), np.linspace(num_leaves_min, num_leaves_max, n_possibilities))]
#num_leaves_grid = num_leaves_grid + [100, 50, 75, 125, 11,150]
##print('Num_Leaves_Grid: ')
##print(num_leaves_grid, len(num_leaves_grid))
##print('\n')
#
#
#min_child_samples_grid = [i for i in map(lambda x: int(x), np.linspace(min_child_samples_min, min_child_samples_max, n_possibilities))]
##print('min_child_samples_grid:')
##print(min_child_samples_grid, len(min_child_samples_grid))
##print('\n')
#
#n_estimators_grid = [i for i in map(lambda x: int(x), np.linspace(n_estimators_min, n_estimators_max, n_possibilities))]
##print('n_estimators_grid:')
##print(n_estimators_grid, len(n_estimators_grid))
##print('\n')
#
#learning_rate_grid = np.linspace(learning_rate_min, learning_rate_max, n_possibilities)
##print('learning_rate_grid:')
##print(learning_rate_grid, len(learning_rate_grid))
##print('\n')
#
#subsample_for_bin_grid = [i for i in map(lambda x: int(x), np.linspace(subsample_for_bin_min, subsample_for_bin_max, n_possibilities))]
##print('subsample_for_bin_grid:')
##print(subsample_for_bin_grid, len(subsample_for_bin_grid))
##print('\n')
#
#min_split_gain_grid = np.linspace(min_split_gain_min, min_split_gain_max, n_possibilities)
##print('min_split_gain_grid:')
##print(min_split_gain_grid, len(min_split_gain_grid))
##print('\n')
#
#min_child_weight_grid = np.linspace(min_child_weight_min, min_child_weight_max, n_possibilities)
##print('min_child_weight_grid:')
##print(min_child_weight_grid, len(min_child_weight_grid))
##print('\n')
#
#reg_alpha_grid = np.linspace(reg_alpha_min, reg_alpha_max, n_possibilities)
##print('reg_alpha_grid:')
##print(reg_alpha_grid, len(reg_alpha_grid))
##print('\n')


In [49]:
#parametros = {'num_leaves': num_leaves_grid, #int
#              'min_child_samples': min_child_samples_grid,#int
#              'n_estimators': n_estimators_grid, #int
#              'learning_rate': learning_rate_grid,
#              'subsample_for_bin': subsample_for_bin_grid, # int
#              'min_split_gain': min_split_gain_grid,
#              'min_child_weight': min_child_weight_grid,
#              'reg_alpha': reg_alpha_grid,
#              'max_depth': [-1],
#              'n_jobs': [-1]}

In [50]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, verbose = 3)
#grid_search.fit(x_wine, y_wine)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)

In [51]:
#rid_search.cv_results_.keys()

In [52]:
#trlist = grid_search.cv_results_['mean_test_score']

In [53]:
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [54]:
#tracking_GS_Accuracy = scorelist
#plt.plot(scorelist); 

### F1 Score

In [55]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, scoring='f1', verbose = 3)
#grid_search.fit(x_wine, y_wine)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)
#
#trlist = grid_search.cv_results_['mean_test_score']
#
#
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#tracking_GS_F1 = scorelist
#plt.plot(scorelist); 

### AUC

In [56]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, scoring='roc_auc', verbose = 3)
#grid_search.fit(x_wine, y_wine)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)
#
#trlist = grid_search.cv_results_['mean_test_score']
#
#
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#tracking_GS_AUC = scorelist
#plt.plot(scorelist); 

## Optuna

### Acurácia

In [57]:
#!pip install optuna

In [58]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
import sklearn
from sklearn import datasets
def objective_accuracy(trial):
      
      iterations = trial.suggest_int('iterations', iterations_min, iterations_max)  
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)  
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', l2_leaf_reg_min, l2_leaf_reg_max)  
      bagging_temperature = trial.suggest_float('bagging_temperature', bagging_temperature_min, bagging_temperature_max)  
      random_strength = trial.suggest_float('random_strength', random_strength_min, random_strength_max)  
      best_model_min_trees = trial.suggest_int('best_model_min_trees', best_model_min_trees_min, best_model_min_trees_max)  
      depth = trial.suggest_int('depth', depth_min, depth_max)  
      min_data_in_leaf = trial.suggest_int('min_data_in_leaf', min_data_in_leaf_min, min_data_in_leaf_max)  
      max_leaves = trial.suggest_int('max_leaves', max_leaves_min, max_leaves_max)  
      one_hot_max_size = trial.suggest_int('one_hot_max_size', one_hot_max_size_min, one_hot_max_size_max)  
      rsm = trial.suggest_float('rsm', rsm_min, rsm_max)  
      fold_permutation_block = trial.suggest_int('fold_permutation_block', fold_permutation_block_min, fold_permutation_block_max)  
      leaf_estimation_iterations = trial.suggest_int('leaf_estimation_iterations', leaf_estimation_iterations_min, leaf_estimation_iterations_max)  
      fold_len_multiplier = trial.suggest_float('fold_len_multiplier', fold_len_multiplier_min, fold_len_multiplier_max)  
      boost_from_average = trial.suggest_int('boost_from_average', 0, 1)  
      posterior_sampling = trial.suggest_int('posterior_sampling', 0, 1)  
      allow_const_label = trial.suggest_int('allow_const_label', 0, 1)  
      penalties_coefficient = trial.suggest_float('penalties_coefficient', penalties_coefficient_min, penalties_coefficient_max)  
      model_shrink_rate = trial.suggest_float('model_shrink_rate', model_shrink_rate_min, model_shrink_rate_max)  
      
      
      model_catboost = CatBoostClassifier(
                                    iterations = iterations,
                                    learning_rate = learning_rate,
                                    l2_leaf_reg = l2_leaf_reg,
                                    bagging_temperature = bagging_temperature,
                                    random_strength = random_strength,
                                    best_model_min_trees = best_model_min_trees,
                                    depth = depth,
                                    min_data_in_leaf = min_data_in_leaf,
                                    max_leaves = max_leaves,
                                    one_hot_max_size = one_hot_max_size,
                                    rsm = rsm,
                                    fold_permutation_block = fold_permutation_block,
                                    leaf_estimation_iterations = leaf_estimation_iterations,
                                    fold_len_multiplier = fold_len_multiplier,
                                    #boost_from_average = bool(boost_from_average),
                                    posterior_sampling = bool(posterior_sampling),
                                    allow_const_label = bool(allow_const_label),
                                    penalties_coefficient = penalties_coefficient,
                                    model_shrink_rate = model_shrink_rate,
                                    approx_on_full_history = False,
                                    loss_function='MultiClass',
                                    grow_policy = 'Lossguide',
                                    thread_count = -1
                                    )

      
      kfold = KFold(n_splits = 3, shuffle = True)

      return sklearn.model_selection.cross_val_score(model_catboost, x_wine, y_wine, n_jobs=-1, cv=kfold).mean()

ModuleNotFoundError: No module named 'optuna'

In [ ]:
def run_optuna_accuracy(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_accuracy, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('Accuracy: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study


In [ ]:
trial, study = run_optuna_accuracy(1)

In [ ]:
trial

0.21775305262434938

In [ ]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)

In [ ]:
#tracking_Optuna_Accuracy = tracking

In [ ]:
#plt.plot(tracking);
#plt.plot(tracking_sorted) 

In [ ]:
#optuna.visualization.plot_optimization_history(study)

In [ ]:
#optuna.visualization.plot_slice(study)

### F1 Score

In [ ]:
def objective_f1(trial):
      iterations = trial.suggest_int('iterations', iterations_min, iterations_max)  
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)  
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', l2_leaf_reg_min, l2_leaf_reg_max)  
      bagging_temperature = trial.suggest_float('bagging_temperature', bagging_temperature_min, bagging_temperature_max)  
      random_strength = trial.suggest_float('random_strength', random_strength_min, random_strength_max)  
      best_model_min_trees = trial.suggest_int('best_model_min_trees', best_model_min_trees_min, best_model_min_trees_max)  
      depth = trial.suggest_int('depth', depth_min, depth_max)  
      min_data_in_leaf = trial.suggest_int('min_data_in_leaf', min_data_in_leaf_min, min_data_in_leaf_max)  
      max_leaves = trial.suggest_int('max_leaves', max_leaves_min, max_leaves_max)  
      one_hot_max_size = trial.suggest_int('one_hot_max_size', one_hot_max_size_min, one_hot_max_size_max)  
      rsm = trial.suggest_float('rsm', rsm_min, rsm_max)  
      fold_permutation_block = trial.suggest_int('fold_permutation_block', fold_permutation_block_min, fold_permutation_block_max)  
      leaf_estimation_iterations = trial.suggest_int('leaf_estimation_iterations', leaf_estimation_iterations_min, leaf_estimation_iterations_max)  
      fold_len_multiplier = trial.suggest_float('fold_len_multiplier', fold_len_multiplier_min, fold_len_multiplier_max)  
      boost_from_average = trial.suggest_int('boost_from_average', 0, 1)  
      posterior_sampling = trial.suggest_int('posterior_sampling', 0, 1)  
      allow_const_label = trial.suggest_int('allow_const_label', 0, 1)  
      penalties_coefficient = trial.suggest_float('penalties_coefficient', penalties_coefficient_min, penalties_coefficient_max)  
      model_shrink_rate = trial.suggest_float('model_shrink_rate', model_shrink_rate_min, model_shrink_rate_max)  
                                                                
      
      model_catboost = CatBoostClassifier(
                                    iterations = iterations,
                                    learning_rate = learning_rate,
                                    l2_leaf_reg = l2_leaf_reg,
                                    
                                    bagging_temperature = bagging_temperature,
                                    random_strength = random_strength,
                                    best_model_min_trees = best_model_min_trees,
                                    depth = depth,
                                    
                                    min_data_in_leaf = min_data_in_leaf,
                                    max_leaves = max_leaves,
                                    one_hot_max_size = one_hot_max_size,
                                    
                                    rsm = rsm,
                                    fold_permutation_block = fold_permutation_block,
                                    leaf_estimation_iterations = leaf_estimation_iterations,
                                    
                                    
                                    fold_len_multiplier = fold_len_multiplier, #Flag Error
                                    #boost_from_average = bool(boost_from_average), #Flag Error
                                    posterior_sampling = bool(posterior_sampling),#Flag Error
                                    
                                    allow_const_label = bool(allow_const_label),
                                    penalties_coefficient = penalties_coefficient,
                                    model_shrink_rate = model_shrink_rate,
                                    approx_on_full_history = False,
                                    
                                    grow_policy = 'Lossguide',
                                    loss_function='MultiClass',
                                    thread_count = -1
                                    )

      
      kfold = KFold(n_splits = 3, shuffle = True)


      return sklearn.model_selection.cross_val_score(model_catboost, x_wine, y_wine, n_jobs=-1, scoring='f1_weighted', cv=kfold).mean()

In [ ]:
def run_optuna_f1(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_f1, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('F1: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study

In [ ]:
trial, study = run_optuna_f1(10)

In [ ]:
trial

0.08598495569493116

In [ ]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)
#
#tracking_Optuna_F1 = tracking
#
#plt.plot(tracking);
##plt.plot(tracking_sorted) 

### AUC

In [ ]:
def objective_auc(trial):
  
      iterations = trial.suggest_int('iterations', iterations_min, iterations_max)  
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)  
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', l2_leaf_reg_min, l2_leaf_reg_max)  
      bagging_temperature = trial.suggest_float('bagging_temperature', bagging_temperature_min, bagging_temperature_max)  
      random_strength = trial.suggest_float('random_strength', random_strength_min, random_strength_max)  
      best_model_min_trees = trial.suggest_int('best_model_min_trees', best_model_min_trees_min, best_model_min_trees_max)  
      depth = trial.suggest_int('depth', depth_min, depth_max)  
      min_data_in_leaf = trial.suggest_int('min_data_in_leaf', min_data_in_leaf_min, min_data_in_leaf_max)  
      max_leaves = trial.suggest_int('max_leaves', max_leaves_min, max_leaves_max)  
      one_hot_max_size = trial.suggest_int('one_hot_max_size', one_hot_max_size_min, one_hot_max_size_max)  
      rsm = trial.suggest_float('rsm', rsm_min, rsm_max)  
      fold_permutation_block = trial.suggest_int('fold_permutation_block', fold_permutation_block_min, fold_permutation_block_max)  
      leaf_estimation_iterations = trial.suggest_int('leaf_estimation_iterations', leaf_estimation_iterations_min, leaf_estimation_iterations_max)  
      fold_len_multiplier = trial.suggest_float('fold_len_multiplier', fold_len_multiplier_min, fold_len_multiplier_max)  
      boost_from_average = trial.suggest_int('boost_from_average', 0, 1)  
      posterior_sampling = trial.suggest_int('posterior_sampling', 0, 1)  
      allow_const_label = trial.suggest_int('allow_const_label', 0, 1)  
      penalties_coefficient = trial.suggest_float('penalties_coefficient', penalties_coefficient_min, penalties_coefficient_max)  
      model_shrink_rate = trial.suggest_float('model_shrink_rate', model_shrink_rate_min, model_shrink_rate_max)  
                                                                
      
      model_catboost = CatBoostClassifier(
                                    iterations = iterations,
                                    learning_rate = learning_rate,
                                    l2_leaf_reg = l2_leaf_reg,
                                    
                                    bagging_temperature = bagging_temperature,
                                    random_strength = random_strength,
                                    best_model_min_trees = best_model_min_trees,
                                    depth = depth,
                                    
                                    min_data_in_leaf = min_data_in_leaf,
                                    max_leaves = max_leaves,
                                    one_hot_max_size = one_hot_max_size,
                                    
                                    rsm = rsm,
                                    fold_permutation_block = fold_permutation_block,
                                    leaf_estimation_iterations = leaf_estimation_iterations,
                                    
                                    
                                    fold_len_multiplier = fold_len_multiplier, #Flag Error
                                    #boost_from_average = bool(boost_from_average), #Flag Error
                                    posterior_sampling = bool(posterior_sampling),#Flag 
                                    
                                    allow_const_label = bool(allow_const_label),
                                    penalties_coefficient = penalties_coefficient,
                                    model_shrink_rate = model_shrink_rate,
                                    approx_on_full_history = False,
                                    
                                    grow_policy = 'Lossguide',
                                    loss_function='MultiClass',
                                    thread_count = -1
                                    )

      
      kfold = KFold(n_splits = 3, shuffle = True)


      return sklearn.model_selection.cross_val_score(model_catboost, x_wine, y_wine, n_jobs=-1, scoring='roc_auc_ovr_weighted', cv=kfold).mean()

In [ ]:
def run_optuna_auc(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_auc, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('AUC: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study

In [ ]:
trial, study = run_optuna_auc(10)

In [ ]:
trial

0.5

In [ ]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)
#
#tracking_Optuna_AUC = tracking
#
#plt.plot(tracking);
##plt.plot(tracking_sorted) 

# Análise Comparativa

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
#
#ax1.set_title('PSO X Gerações')
#ax2.set_title('PSO X Avaliações')
#
#ax1.plot(tracking_PSO_Accuracy)
#ax2.plot(Accuracy_PSO)
#plt.show()

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
#
#ax1.set_title('GA X Gerações')
#ax2.set_title('GA X Avaliações')
#
#ax1.plot(tracking_GA_Accuracy)
#ax2.plot(Accuracy_GA)
#plt.show()

Gráficos Comparativos Acurácia

In [ ]:
#from IPython.core.pylabtools import figsize
#from matplotlib.pyplot import figure
#import matplotlib.patches as mpatches
#
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(Accuracy_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(Accuracy_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_Accuracy, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_Accuracy, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_Accuracy, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_Accuracy, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_Accuracy, color = 'green', linewidth=3, linestyle='-')


F1 score

In [ ]:
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(F1_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(F1_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_F1, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_F1, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_F1, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_F1, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_F1, color = 'green', linewidth=3, linestyle='-')


ROC AUC

In [ ]:
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(AUC_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(AUC_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_AUC, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_AUC, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_AUC, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_AUC, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_AUC, color = 'green', linewidth=3, linestyle='-')
#

# Random Search

### Accuracy

In [ ]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))


def run_random_search_accuracy(iterations = 1024):
    param_distribution = {
    'iterations' : random_search_int_range(iterations_min, iterations_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'l2_leaf_reg' : uniform(l2_leaf_reg_min, l2_leaf_reg_max),
    'bagging_temperature' : uniform(bagging_temperature_min, bagging_temperature_max),
    'random_strength' : uniform(random_strength_min, random_strength_max),
    'best_model_min_trees' : random_search_int_range(best_model_min_trees_min, best_model_min_trees_max),
    'depth' : random_search_int_range(depth_min, depth_max),
    'min_data_in_leaf' : random_search_int_range(min_data_in_leaf_min, min_data_in_leaf_max),
    'max_leaves' : random_search_int_range(max_leaves_min, max_leaves_max),
    'one_hot_max_size' : random_search_int_range(one_hot_max_size_min, one_hot_max_size_max),
    'rsm' : uniform(rsm_min + 0.00001, rsm_max),
    'fold_permutation_block' : random_search_int_range(fold_permutation_block_min, fold_permutation_block_max),
    'leaf_estimation_iterations' : random_search_int_range(leaf_estimation_iterations_min, leaf_estimation_iterations_max),
    'fold_len_multiplier' : uniform(fold_len_multiplier_min, fold_len_multiplier_max),
    'posterior_sampling' : [posterior_sampling_min, posterior_sampling_max],
    'allow_const_label' : [allow_const_label_min, allow_const_label_max],
    'penalties_coefficient' : uniform(penalties_coefficient_min, penalties_coefficient_max),
    'model_shrink_rate' : uniform(model_shrink_rate_min, model_shrink_rate_max),
    'approx_on_full_history' : [False],
    'grow_policy' : ['Lossguide'],
    'loss_function' : ['MultiClass'],
    'thread_count' :  [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(CatBoostClassifier(), param_distribution, n_iter = iterations, n_jobs = -1, cv=kfold, verbose=False)
    random_search_accuracy.fit(x_wine, y_wine)

    return random_search_accuracy.best_score_

In [ ]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))


def run_random_search_f1(iterations = 1024):
    param_distribution = {
    'iterations' : random_search_int_range(iterations_min, iterations_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'l2_leaf_reg' : uniform(l2_leaf_reg_min, l2_leaf_reg_max),
    'bagging_temperature' : uniform(bagging_temperature_min, bagging_temperature_max),
    'random_strength' : uniform(random_strength_min, random_strength_max),
    'best_model_min_trees' : random_search_int_range(best_model_min_trees_min, best_model_min_trees_max),
    'depth' : random_search_int_range(depth_min, depth_max),
    'min_data_in_leaf' : random_search_int_range(min_data_in_leaf_min, min_data_in_leaf_max),
    'max_leaves' : random_search_int_range(max_leaves_min, max_leaves_max),
    'one_hot_max_size' : random_search_int_range(one_hot_max_size_min, one_hot_max_size_max),
    'rsm' : uniform(rsm_min + 0.00001, rsm_max),
    'fold_permutation_block' : random_search_int_range(fold_permutation_block_min, fold_permutation_block_max),
    'leaf_estimation_iterations' : random_search_int_range(leaf_estimation_iterations_min, leaf_estimation_iterations_max),
    'fold_len_multiplier' : uniform(fold_len_multiplier_min, fold_len_multiplier_max),
    'posterior_sampling' : [posterior_sampling_min, posterior_sampling_max],
    'allow_const_label' : [allow_const_label_min, allow_const_label_max],
    'penalties_coefficient' : uniform(penalties_coefficient_min, penalties_coefficient_max),
    'model_shrink_rate' : uniform(model_shrink_rate_min, model_shrink_rate_max),
    'approx_on_full_history' : [False],
    'grow_policy' : ['Lossguide'],
    'loss_function' : ['MultiClass'],
    'thread_count' :  [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(CatBoostClassifier(), param_distribution, n_iter = iterations, n_jobs = -1, cv=kfold, verbose=False, scoring='f1_weighted')
    random_search_accuracy.fit(x_wine, y_wine)

    return random_search_accuracy.best_score_

In [ ]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))


def run_random_search_auc(iterations = 1024):
    param_distribution = {
    'iterations' : random_search_int_range(iterations_min, iterations_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'l2_leaf_reg' : uniform(l2_leaf_reg_min, l2_leaf_reg_max),
    'bagging_temperature' : uniform(bagging_temperature_min, bagging_temperature_max),
    'random_strength' : uniform(random_strength_min, random_strength_max),
    'best_model_min_trees' : random_search_int_range(best_model_min_trees_min, best_model_min_trees_max),
    'depth' : random_search_int_range(depth_min, depth_max),
    'min_data_in_leaf' : random_search_int_range(min_data_in_leaf_min, min_data_in_leaf_max),
    'max_leaves' : random_search_int_range(max_leaves_min, max_leaves_max),
    'one_hot_max_size' : random_search_int_range(one_hot_max_size_min, one_hot_max_size_max),
    'rsm' : uniform(rsm_min + 0.00001, rsm_max),
    'fold_permutation_block' : random_search_int_range(fold_permutation_block_min, fold_permutation_block_max),
    'leaf_estimation_iterations' : random_search_int_range(leaf_estimation_iterations_min, leaf_estimation_iterations_max),
    'fold_len_multiplier' : uniform(fold_len_multiplier_min, fold_len_multiplier_max),
    'posterior_sampling' : [posterior_sampling_min, posterior_sampling_max],
    'allow_const_label' : [allow_const_label_min, allow_const_label_max],
    'penalties_coefficient' : uniform(penalties_coefficient_min, penalties_coefficient_max),
    'model_shrink_rate' : uniform(model_shrink_rate_min, model_shrink_rate_max),
    'approx_on_full_history' : [False],
    'grow_policy' : ['Lossguide'],
    'loss_function' : ['MultiClass'],
    'thread_count' :  [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(CatBoostClassifier(), param_distribution, n_iter = iterations, n_jobs = -1, cv=kfold, verbose=False, scoring='roc_auc_ovr_weighted')
    random_search_accuracy.fit(x_wine, y_wine)

    return random_search_accuracy.best_score_

# Gerando Dados Para Análise 

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run(iterations):
    filename = './Catboost_wine_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):
        accuracy_pso, _, _ = run_accuracy_pso()
        f1_pso, _, _ = run_f1_pso()
        auc_pso, _, _ = run_auc_pso()
        temp = pd.DataFrame({'Algorithm' : ['PSO'], 
                            'Accuracy' :[-accuracy_pso[0]], 
                            'F1' : [-f1_pso[0]], 
                            'AUC' : [-auc_pso[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        
        accuracy_ga, _, _ = run_accuracy_ga()
        f1_ga, _, _ = run_f1_ga()
        auc_ga, _, _ = run_auc_ga()
        temp = pd.DataFrame({'Algorithm' : ['GA'], 
                            'Accuracy' :[-accuracy_ga[0]], 
                            'F1' : [-f1_ga[0]], 
                            'AUC' : [-auc_ga[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        
        accuracy_optuna, _ = run_optuna_accuracy()
        f1_optuna, _ = run_optuna_f1()
        auc_optuna, _ = run_optuna_auc()
        temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
                            'Accuracy' :[accuracy_optuna], 
                            'F1' : [f1_optuna], 
                            'AUC' : [auc_optuna]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_pso(iterations):
    filename = './Catboost_wine_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):
        accuracy_pso, _, _ = run_accuracy_pso()
        f1_pso, _, _ = run_f1_pso()
        auc_pso, _, _ = run_auc_pso()
        temp = pd.DataFrame({'Algorithm' : ['PSO'], 
                            'Accuracy' :[-accuracy_pso[0]], 
                            'F1' : [-f1_pso[0]], 
                            'AUC' : [-auc_pso[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_ga(iterations):
    filename = './Catboost_wine_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):

        accuracy_ga, _, _ = run_accuracy_ga()
        f1_ga, _, _ = run_f1_ga()
        auc_ga, _, _ = run_auc_ga()
        temp = pd.DataFrame({'Algorithm' : ['GA'], 
                            'Accuracy' :[-accuracy_ga[0]], 
                            'F1' : [-f1_ga[0]], 
                            'AUC' : [-auc_ga[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_optuna(iterations):
    filename = './Catboost_wine_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):      
        accuracy_optuna, _ = run_optuna_accuracy()
        f1_optuna, _ = run_optuna_f1()
        auc_optuna, _ = run_optuna_auc()
        temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
                            'Accuracy' :[accuracy_optuna], 
                            'F1' : [f1_optuna], 
                            'AUC' : [auc_optuna]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [ ]:
def full_run_random_searh(iterations):
    filename = './Catboost_wine_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):

        accuracy_rs = run_random_search_accuracy()
        f1_rs = run_random_search_f1()
        auc_rs = run_random_search_auc()
        temp = pd.DataFrame({'Algorithm' : ['Random Search'], 
                            'Accuracy' :[accuracy_rs], 
                            'F1' : [f1_rs], 
                            'AUC' : [auc_rs]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [ ]:
full_run_random_searh(1)

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

0:	learn: 1.5847722	total: 12.6ms	remaining: 1.12s
1:	learn: 1.5825045	total: 25.2ms	remaining: 1.11s
2:	learn: 1.5620491	total: 37.5ms	remaining: 1.09s
3:	learn: 1.5284127	total: 50ms	remaining: 1.07s
4:	learn: 1.4736686	total: 62.4ms	remaining: 1.06s
5:	learn: 1.4626588	total: 74.9ms	remaining: 1.05s
6:	learn: 1.4430127	total: 86.9ms	remaining: 1.03s
7:	learn: 1.4259507	total: 99.1ms	remaining: 1.01s
8:	learn: 1.3591770	total: 111ms	remaining: 1s
9:	learn: 1.3524211	total: 123ms	remaining: 985ms
10:	learn: 1.3477283	total: 135ms	remaining: 970ms
11:	learn: 1.3461622	total: 147ms	remaining: 956ms
12:	learn: 1.3453215	total: 159ms	remaining: 942ms
13:	learn: 1.3396274	total: 171ms	remaining: 929ms
14:	learn: 1.3322868	total: 184ms	remaining: 918ms
15:	learn: 1.3068390	total: 195ms	remaining: 904ms
16:	learn: 1.2849474	total: 208ms	remaining: 893ms
17:	learn: 1.2778638	total: 221ms	remaining: 882ms
18:	learn: 1.2759855	total: 233ms	remaining: 872ms
19:	learn: 1.2655587	total: 246ms	rema

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.07402653 0.07402653 0.07402653 ... 0.07402653 0.07402653 0.07402653]
  warnings.warn(


0:	learn: 1.5710101	total: 4.63ms	remaining: 366ms
1:	learn: 1.5611104	total: 9.08ms	remaining: 354ms
2:	learn: 1.4859781	total: 14.3ms	remaining: 366ms
3:	learn: 1.4844660	total: 19.6ms	remaining: 373ms
4:	learn: 1.4622307	total: 24.9ms	remaining: 373ms
5:	learn: 1.3754553	total: 29.9ms	remaining: 369ms
6:	learn: 1.3716098	total: 34.9ms	remaining: 364ms
7:	learn: 1.3610626	total: 40.1ms	remaining: 360ms
8:	learn: 1.3546364	total: 46.5ms	remaining: 367ms
9:	learn: 1.3546114	total: 51.9ms	remaining: 364ms
10:	learn: 1.3414571	total: 56.8ms	remaining: 356ms
11:	learn: 1.3397835	total: 61.4ms	remaining: 348ms
12:	learn: 1.3147731	total: 66.5ms	remaining: 343ms
13:	learn: 1.3030325	total: 71.6ms	remaining: 337ms
14:	learn: 1.3005668	total: 76.6ms	remaining: 332ms
15:	learn: 1.2982224	total: 81.9ms	remaining: 327ms
16:	learn: 1.2945915	total: 86.7ms	remaining: 321ms
17:	learn: 1.2946078	total: 91.5ms	remaining: 315ms
18:	learn: 1.2912234	total: 96.4ms	remaining: 310ms
19:	learn: 1.2874025	t

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.5 0.5 0.5 ... 0.5 0.5 0.5]
  warnings.warn(


0:	learn: 1.5928194	total: 6.1ms	remaining: 153ms
1:	learn: 1.5543222	total: 12.3ms	remaining: 147ms
2:	learn: 1.5368320	total: 18.3ms	remaining: 140ms
3:	learn: 1.5031061	total: 24.5ms	remaining: 135ms
4:	learn: 1.4519779	total: 30.7ms	remaining: 129ms
5:	learn: 1.4187094	total: 37ms	remaining: 123ms
6:	learn: 1.4173160	total: 43.5ms	remaining: 118ms
7:	learn: 1.3669621	total: 49.7ms	remaining: 112ms
8:	learn: 1.3507530	total: 56.1ms	remaining: 106ms
9:	learn: 1.3444820	total: 62.6ms	remaining: 100ms
10:	learn: 1.3305004	total: 68.9ms	remaining: 93.9ms
11:	learn: 1.3229321	total: 75.1ms	remaining: 87.6ms
12:	learn: 1.3135686	total: 81.8ms	remaining: 81.8ms
13:	learn: 1.3071690	total: 88.3ms	remaining: 75.7ms
14:	learn: 1.3035841	total: 94.5ms	remaining: 69.3ms
15:	learn: 1.2986790	total: 101ms	remaining: 62.8ms
16:	learn: 1.2950433	total: 107ms	remaining: 56.5ms
17:	learn: 1.2931443	total: 113ms	remaining: 50.1ms
18:	learn: 1.2895920	total: 119ms	remaining: 43.8ms
19:	learn: 1.2805065

In [ ]:
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
6 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

0:	learn: 1.6004375	total: 12.3ms	remaining: 897ms
1:	learn: 1.5802098	total: 23.8ms	remaining: 856ms
2:	learn: 1.5802229	total: 34.6ms	remaining: 820ms
3:	learn: 1.5795879	total: 45.8ms	remaining: 802ms
4:	learn: 1.5746869	total: 57.3ms	remaining: 790ms
5:	learn: 1.5490451	total: 78.6ms	remaining: 891ms
6:	learn: 1.5497388	total: 89.8ms	remaining: 860ms
7:	learn: 1.5258609	total: 111ms	remaining: 917ms
8:	learn: 1.5174868	total: 122ms	remaining: 883ms
9:	learn: 1.5148800	total: 134ms	remaining: 856ms
10:	learn: 1.5159483	total: 145ms	remaining: 829ms
11:	learn: 1.5053703	total: 156ms	remaining: 806ms
12:	learn: 1.5015169	total: 167ms	remaining: 785ms
13:	learn: 1.4882557	total: 179ms	remaining: 767ms
14:	learn: 1.4849708	total: 190ms	remaining: 748ms
15:	learn: 1.4854914	total: 201ms	remaining: 729ms
16:	learn: 1.4732359	total: 214ms	remaining: 718ms
17:	learn: 1.4730484	total: 225ms	remaining: 700ms
18:	learn: 1.4750217	total: 236ms	remaining: 683ms
19:	learn: 1.4747489	total: 248ms	

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

0:	learn: 1.5803804	total: 9.6ms	remaining: 711ms
1:	learn: 1.5672609	total: 18.5ms	remaining: 677ms
2:	learn: 1.5610595	total: 27.9ms	remaining: 671ms
3:	learn: 1.5580051	total: 37.3ms	remaining: 662ms
4:	learn: 1.5506275	total: 46.6ms	remaining: 653ms
5:	learn: 1.4818379	total: 56.5ms	remaining: 649ms
6:	learn: 1.4667562	total: 66ms	remaining: 642ms
7:	learn: 1.3804120	total: 75.7ms	remaining: 634ms
8:	learn: 1.3713480	total: 84.8ms	remaining: 622ms
9:	learn: 1.3674283	total: 94ms	remaining: 611ms
10:	learn: 1.3649205	total: 103ms	remaining: 600ms
11:	learn: 1.3526817	total: 112ms	remaining: 589ms
12:	learn: 1.2986242	total: 122ms	remaining: 582ms
13:	learn: 1.2964169	total: 132ms	remaining: 573ms
14:	learn: 1.2784561	total: 141ms	remaining: 565ms
15:	learn: 1.2540590	total: 150ms	remaining: 555ms
16:	learn: 1.2519464	total: 160ms	remaining: 546ms
17:	learn: 1.2461609	total: 169ms	remaining: 536ms
18:	learn: 1.2439302	total: 179ms	remaining: 528ms
19:	learn: 1.2420244	total: 190ms	re

...

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

0:	learn: 1.5782087	total: 10.1ms	remaining: 674ms
1:	learn: 1.4563508	total: 20.1ms	remaining: 665ms
2:	learn: 1.4164612	total: 30.6ms	remaining: 664ms
3:	learn: 1.3711996	total: 41ms	remaining: 656ms
4:	learn: 1.3382142	total: 51.2ms	remaining: 645ms
5:	learn: 1.3028556	total: 61.4ms	remaining: 635ms
6:	learn: 1.2968057	total: 71.4ms	remaining: 622ms
7:	learn: 1.2938427	total: 81.7ms	remaining: 613ms
8:	learn: 1.2838804	total: 91.9ms	remaining: 602ms
9:	learn: 1.2837326	total: 102ms	remaining: 591ms
10:	learn: 1.2772281	total: 112ms	remaining: 581ms
11:	learn: 1.2626626	total: 122ms	remaining: 571ms
12:	learn: 1.2395133	total: 133ms	remaining: 562ms
13:	learn: 1.2249607	total: 143ms	remaining: 550ms
14:	learn: 1.2244518	total: 153ms	remaining: 539ms
15:	learn: 1.2044126	total: 163ms	remaining: 529ms
16:	learn: 1.1788153	total: 173ms	remaining: 518ms
17:	learn: 1.1700117	total: 183ms	remaining: 508ms
18:	learn: 1.1654503	total: 194ms	remaining: 499ms
19:	learn: 1.1524586	total: 204ms	

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
6 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

0:	learn: 1.5694518	total: 11.6ms	remaining: 904ms
1:	learn: 1.5511396	total: 23ms	remaining: 884ms
2:	learn: 1.4183644	total: 34.6ms	remaining: 876ms
3:	learn: 1.4142263	total: 46ms	remaining: 862ms
4:	learn: 1.4087921	total: 57.3ms	remaining: 847ms
5:	learn: 1.4061138	total: 68.4ms	remaining: 832ms
6:	learn: 1.3921038	total: 79.8ms	remaining: 821ms
7:	learn: 1.2724303	total: 90.9ms	remaining: 807ms
8:	learn: 1.2455488	total: 102ms	remaining: 793ms
9:	learn: 1.2421891	total: 113ms	remaining: 782ms
10:	learn: 1.2338892	total: 125ms	remaining: 770ms
11:	learn: 1.2258676	total: 136ms	remaining: 759ms
12:	learn: 1.2230680	total: 147ms	remaining: 749ms
13:	learn: 1.1935345	total: 159ms	remaining: 738ms
14:	learn: 1.1867422	total: 170ms	remaining: 726ms
15:	learn: 1.1855458	total: 182ms	remaining: 716ms
16:	learn: 1.1821906	total: 194ms	remaining: 709ms
17:	learn: 1.1772956	total: 206ms	remaining: 698ms
18:	learn: 1.1724010	total: 218ms	remaining: 688ms
19:	learn: 1.1530292	total: 230ms	rem